# LLM Optimization Modelling Experiment

In [4]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [62]:
problem = '''Consider a farmer who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop.

The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler.
Any production in excess of the feeding requirement would be sold.

Over the last decade, mean selling prices have been $170 and $150 per ton of wheat and corn, respectively.
The purchase prices are 40% more than this due to the wholesaler’s margin and transportation costs.

Another profitable crop is sugar beet, which she expects to sell at $36/T. However, the European Commission imposes a quota on sugar beet production. Any amount in excess of the quota can be sold only at $10/T. The farmer’s quota for next year is 6000 T.

Based on past experience, the farmer knows that the mean yield on her land is roughly 2.5 T, 3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. It costs $150 to plant an acre of wheat, $230 to pant an acre of corn and $260 to plant an acre of sugar beets. 

The farmer can decide how much of her land she will use to grow each of the three products: wheat, corn and sugar beets. The goal of the farmer is to minimize her total costs (where sales are considered as negative costs).'''

## 2. Generate the mathematical model

In [63]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [69]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Farmer's Crop Planning

**Parameters:**

* $A$: Total land available (500 acres)
* $W_{demand}$: Wheat demand for cattle feed (200 T)
* $C_{demand}$: Corn demand for cattle feed (240 T)
* $S_{quota}$: Sugar beet quota (6000 T)
* $Y_w$: Yield of wheat per acre (2.5 T/acre)
* $Y_c$: Yield of corn per acre (3 T/acre)
* $Y_s$: Yield of sugar beet per acre (20 T/acre)
* $SP_w$: Selling price of wheat ($170/T)
* $SP_c$: Selling price of corn ($150/T)
* $SP_{s1}$: Selling price of sugar beet within quota ($36/T)
* $SP_{s2}$: Selling price of sugar beet exceeding quota ($10/T)
* $PP_w$: Purchase price of wheat ($170 * 1.4 = $238/T)
* $PP_c$: Purchase price of corn ($150 * 1.4 = $210/T)
* $PC_w$: Planting cost of wheat ($150/acre)
* $PC_c$: Planting cost of corn ($230/acre)
* $PC_s$: Planting cost of sugar beet ($260/acre)

**Decision Variables:**

* $A_w$: Acres of land allocated to wheat
* $A_c$: Acres of land allocated to corn
* $A_s$: Acres of land allocated to sugar beet
* $W_{buy}$: Tons of wheat purchased
* $C_{buy}$: Tons of corn purchased
* $S_{sell1}$: Tons of sugar beet sold within quota
* $S_{sell2}$: Tons of sugar beet sold exceeding quota

**Objective Function:**

Minimize total cost:

$Minimize: PC_w * A_w + PC_c * A_c + PC_s * A_s + PP_w * W_{buy} + PP_c * C_{buy} - SP_w * (Y_w * A_w - W_{buy}) - SP_c * (Y_c * A_c - C_{buy}) - SP_{s1} * S_{sell1} - SP_{s2} * S_{sell2}$

**Constraints:**

1. **Land constraint:** $A_w + A_c + A_s \le A$
2. **Wheat production and purchase:** $Y_w * A_w + W_{buy} \ge W_{demand}$
3. **Corn production and purchase:** $Y_c * A_c + C_{buy} \ge C_{demand}$
4. **Sugar beet sales within quota:** $S_{sell1} \le S_{quota}$
5. **Sugar beet production and sales:** $Y_s * A_s = S_{sell1} + S_{sell2}$
6. **Non-negativity:** 
   * $A_w, A_c, A_s \ge 0$
   * $W_{buy}, C_{buy}, S_{sell1}, S_{sell2} \ge 0$ 

This model represents the farmer's problem, aiming to minimize her costs while meeting the demand for cattle feed and respecting the sugar beet quota. The farmer can use this model to find the optimal allocation of land for each crop and the optimal quantities of wheat and corn to purchase. 


## 3. Generate the pyomo code

In [65]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [70]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Parameters
model.A = pyo.Param(initialize=500)  # Total land available
model.W_demand = pyo.Param(initialize=200)  # Wheat demand
model.C_demand = pyo.Param(initialize=240)  # Corn demand
model.S_quota = pyo.Param(initialize=6000)  # Sugar beet quota
model.Y_w = pyo.Param(initialize=2.5)  # Wheat yield
model.Y_c = pyo.Param(initialize=3)  # Corn yield
model.Y_s = pyo.Param(initialize=20)  # Sugar beet yield
model.SP_w = pyo.Param(initialize=170)  # Wheat selling price
model.SP_c = pyo.Param(initialize=150)  # Corn selling price
model.SP_s1 = pyo.Param(initialize=36)  # Sugar beet selling price within quota
model.SP_s2 = pyo.Param(initialize=10)  # Sugar beet selling price exceeding quota
model.PP_w = pyo.Param(initialize=238)  # Wheat purchase price
model.PP_c = pyo.Param(initialize=210)  # Corn purchase price
model.PC_w = pyo.Param(initialize=150)  # Wheat planting cost
model.PC_c = pyo.Param(initialize=230)  # Corn planting cost
model.PC_s = pyo.Param(initialize=260)  # Sugar beet planting cost

# Decision Variables
model.A_w = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land allocated to wheat
model.A_c = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land allocated to corn
model.A_s = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land allocated to sugar beet
model.W_buy = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of wheat purchased
model.C_buy = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of corn purchased
model.S_sell1 = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of sugar beet sold within quota
model.S_sell2 = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of sugar beet sold exceeding quota

# Objective Function
def objective_rule(model):
    return model.PC_w * model.A_w + model.PC_c * model.A_c + model.PC_s * model.A_s + model.PP_w * model.W_buy + model.PP_c * model.C_buy - model.SP_w * (model.Y_w * model.A_w - model.W_buy) - model.SP_c * (model.Y_c * model.A_c - model.C_buy) - model.SP_s1 * model.S_sell1 - model.SP_s2 * model.S_sell2

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.land_constraint = pyo.Constraint(expr=model.A_w + model.A_c + model.A_s <= model.A)
model.wheat_constraint = pyo.Constraint(expr=model.Y_w * model.A_w + model.W_buy >= model.W_demand)
model.corn_constraint = pyo.Constraint(expr=model.Y_c * model.A_c + model.C_buy >= model.C_demand)
model.sugar_quota_constraint = pyo.Constraint(expr=model.S_sell1 <= model.S_quota)
model.sugar_production_constraint = pyo.Constraint(expr=model.Y_s * model.A_s == model.S_sell1 + model.S_sell2)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if you prefer
solver.solve(model)

# Print the results
print("Optimal Solution:")
print(f"Acres of wheat: {pyo.value(model.A_w)}")
print(f"Acres of corn: {pyo.value(model.A_c)}")
print(f"Acres of sugar beet: {pyo.value(model.A_s)}")
print(f"Tons of wheat purchased: {pyo.value(model.W_buy)}")
print(f"Tons of corn purchased: {pyo.value(model.C_buy)}")
print(f"Tons of sugar beet sold within quota: {pyo.value(model.S_sell1)}")
print(f"Tons of sugar beet sold exceeding quota: {pyo.value(model.S_sell2)}")
print(f"Total cost: {pyo.value(model.objective)}")
```

**Sample Data:**

The sample data is defined within the code in the "Parameters" section. For example:

```python
model.A = pyo.Param(initialize=500)  # Total land available
```

This line sets the total land available (`model.A`) to 500 acres. You can modify these values to explore different scenarios. 


## 4. Input problem data and try running the generated code

In [72]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Parameters
model.A = pyo.Param(initialize=500)  # Total land available
model.W_demand = pyo.Param(initialize=200)  # Wheat demand
model.C_demand = pyo.Param(initialize=240)  # Corn demand
model.S_quota = pyo.Param(initialize=6000)  # Sugar beet quota
model.Y_w = pyo.Param(initialize=2.5)  # Wheat yield
model.Y_c = pyo.Param(initialize=3)  # Corn yield
model.Y_s = pyo.Param(initialize=20)  # Sugar beet yield
model.SP_w = pyo.Param(initialize=170)  # Wheat selling price
model.SP_c = pyo.Param(initialize=150)  # Corn selling price
model.SP_s1 = pyo.Param(initialize=36)  # Sugar beet selling price within quota
model.SP_s2 = pyo.Param(initialize=10)  # Sugar beet selling price exceeding quota
model.PP_w = pyo.Param(initialize=238)  # Wheat purchase price
model.PP_c = pyo.Param(initialize=210)  # Corn purchase price
model.PC_w = pyo.Param(initialize=150)  # Wheat planting cost
model.PC_c = pyo.Param(initialize=230)  # Corn planting cost
model.PC_s = pyo.Param(initialize=260)  # Sugar beet planting cost

# Decision Variables
model.A_w = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land allocated to wheat
model.A_c = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land allocated to corn
model.A_s = pyo.Var(domain=pyo.NonNegativeReals)  # Acres of land allocated to sugar beet
model.W_buy = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of wheat purchased
model.C_buy = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of corn purchased
model.S_sell1 = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of sugar beet sold within quota
model.S_sell2 = pyo.Var(domain=pyo.NonNegativeReals)  # Tons of sugar beet sold exceeding quota

# Objective Function
def objective_rule(model):
    return model.PC_w * model.A_w + model.PC_c * model.A_c + model.PC_s * model.A_s + model.PP_w * model.W_buy + model.PP_c * model.C_buy - model.SP_w * (model.Y_w * model.A_w - model.W_buy) - model.SP_c * (model.Y_c * model.A_c - model.C_buy) - model.SP_s1 * model.S_sell1 - model.SP_s2 * model.S_sell2

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.land_constraint = pyo.Constraint(expr=model.A_w + model.A_c + model.A_s <= model.A)
model.wheat_constraint = pyo.Constraint(expr=model.Y_w * model.A_w + model.W_buy >= model.W_demand)
model.corn_constraint = pyo.Constraint(expr=model.Y_c * model.A_c + model.C_buy >= model.C_demand)
model.sugar_quota_constraint = pyo.Constraint(expr=model.S_sell1 <= model.S_quota)
model.sugar_production_constraint = pyo.Constraint(expr=model.Y_s * model.A_s == model.S_sell1 + model.S_sell2)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if you prefer
solver.solve(model)

# Print the results
print("Optimal Solution:")
print(f"Acres of wheat: {pyo.value(model.A_w)}")
print(f"Acres of corn: {pyo.value(model.A_c)}")
print(f"Acres of sugar beet: {pyo.value(model.A_s)}")
print(f"Tons of wheat purchased: {pyo.value(model.W_buy)}")
print(f"Tons of corn purchased: {pyo.value(model.C_buy)}")
print(f"Tons of sugar beet sold within quota: {pyo.value(model.S_sell1)}")
print(f"Tons of sugar beet sold exceeding quota: {pyo.value(model.S_sell2)}")
print(f"Total cost: {pyo.value(model.objective)}")

Optimal Solution:
Acres of wheat: 120.0
Acres of corn: 80.0
Acres of sugar beet: 300.0
Tons of wheat purchased: 0.0
Tons of corn purchased: 0.0
Tons of sugar beet sold within quota: 6000.0
Tons of sugar beet sold exceeding quota: 0.0
Total cost: -188600.0


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [73]:
print(response.text)

## Mathematical Optimization Model for Farmer's Crop Planning

**Parameters:**

* $A$: Total land available (500 acres)
* $W_{demand}$: Wheat demand for cattle feed (200 T)
* $C_{demand}$: Corn demand for cattle feed (240 T)
* $S_{quota}$: Sugar beet quota (6000 T)
* $Y_w$: Yield of wheat per acre (2.5 T/acre)
* $Y_c$: Yield of corn per acre (3 T/acre)
* $Y_s$: Yield of sugar beet per acre (20 T/acre)
* $SP_w$: Selling price of wheat ($170/T)
* $SP_c$: Selling price of corn ($150/T)
* $SP_{s1}$: Selling price of sugar beet within quota ($36/T)
* $SP_{s2}$: Selling price of sugar beet exceeding quota ($10/T)
* $PP_w$: Purchase price of wheat ($170 * 1.4 = $238/T)
* $PP_c$: Purchase price of corn ($150 * 1.4 = $210/T)
* $PC_w$: Planting cost of wheat ($150/acre)
* $PC_c$: Planting cost of corn ($230/acre)
* $PC_s$: Planting cost of sugar beet ($260/acre)

**Decision Variables:**

* $A_w$: Acres of land allocated to wheat
* $A_c$: Acres of land allocated to corn
* $A_s$: Acres of land all

In [74]:
print(response2.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Parameters
model.A = pyo.Param(initialize=500)  # Total land available
model.W_demand = pyo.Param(initialize=200)  # Wheat demand
model.C_demand = pyo.Param(initialize=240)  # Corn demand
model.S_quota = pyo.Param(initialize=6000)  # Sugar beet quota
model.Y_w = pyo.Param(initialize=2.5)  # Wheat yield
model.Y_c = pyo.Param(initialize=3)  # Corn yield
model.Y_s = pyo.Param(initialize=20)  # Sugar beet yield
model.SP_w = pyo.Param(initialize=170)  # Wheat selling price
model.SP_c = pyo.Param(initialize=150)  # Corn selling price
model.SP_s1 = pyo.Param(initialize=36)  # Sugar beet selling price within quota
model.SP_s2 = pyo.Param(initialize=10)  # Sugar beet selling price exceeding quota
model.PP_w = pyo.Param(initialize=238)  # Wheat purchase price
model.PP_c = pyo.Param(initialize=210)  # Corn purchase price
model.PC_w = pyo.Param(initialize=150)  # Wheat planting cost
model.PC_c = pyo.Param(init